## Group3
### Members
1.Andrea Ulloa 
2.Ana Angulo (20171627)
3.Angela Coapaza (20171636) 

# Replication of the PM1_Notebook1_Prediction_newdata using a restricted da

## Introduction

In this wage example, Y is the hourly wage of a worker and X is a vector of worker's characteristics, e.g., education, experience, gender. Two main questions here are:

How to use job-relevant characteristics, such as education and experience, to best predict wages?

What is the difference in predicted wages between men and women with the same job-relevant characteristics?

In this lab, we focus on the prediction question first.

## Data

The data set we consider is from the March Supplement of the U.S. Current Population Survey, year 2015. We select white non-hispanic individuals, aged 25 to 64 years, and working more than 35 hours per week during at least 50 weeks of the year. We exclude self-employed workers; individuals living in group quarters; individuals in the military, agricultural or private household sectors; individuals with inconsistent reports on earnings and employment status; individuals with allocated or missing information in any of the variables used in the analysis; and individuals with hourly wage below  3 .

The variable of interest  Y  is the hourly wage rate constructed as the ratio of the annual earnings to the total number of hours worked, which is constructed in turn as the product of number of weeks worked and the usual number of hours worked per week. In our analysis, we also focus on single (never married) workers. The final sample is of size  **n=5150** .

## Data analysis

We start by loading the data set.

In [18]:
using Pkg


Pkg.add("DataFrames")
Pkg.add("Dates")
Pkg.add("Plots")
Pkg.add("CategoricalArrays")
Pkg.add("RData")
Pkg.add("Lathe")
Pkg.add("GLM") # package to run models 
Pkg.add("StatsPlots")
Pkg.add("MLBase")
Pkg.add("Tables")
Pkg.add("CovarianceMatrices") # robust standar error
Pkg.add("PyPlot")
Pkg.add("CSV")

using DataFrames
using Dates
using Plots
using Statistics,RData  #upload data of R format 
using CategoricalArrays # categorical data
using DataFrames
using Tables
using Lathe
using GLM
using CovarianceMatrices
using StatsPlots
using Statistics

   Resolving package versions...
  No Changes to `C:\Users\acoap\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\acoap\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\acoap\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\acoap\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\acoap\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\acoap\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\acoap\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\acoap\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\acoap\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\acoap\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\acoap\.julia\environments\v1.7\Project.toml`
  No Changes to 

## Then we load some packages that will help us to use the Lasso Metod.

In [3]:
import Pkg; Pkg.add("RData")
import Pkg; Pkg.add("CodecBzip2")
import Pkg; Pkg.add("DataStructures")
import Pkg; Pkg.add("NamedArrays")
import Pkg; Pkg.add("PrettyTables")
import Pkg; Pkg.add("Lasso")

   Resolving package versions...
  No Changes to `C:\Users\acoap\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\acoap\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
   Installed CodecBzip2 ─ v0.7.2
    Updating `C:\Users\acoap\.julia\environments\v1.7\Project.toml`
  [523fee87] + CodecBzip2 v0.7.2
    Updating `C:\Users\acoap\.julia\environments\v1.7\Manifest.toml`
  [523fee87] + CodecBzip2 v0.7.2
Precompiling project...
  ✓ CodecBzip2
  1 dependency successfully precompiled in 1 seconds (193 already precompiled, 1 skipped during auto due to previous errors)
   Resolving package versions...
    Updating `C:\Users\acoap\.julia\environments\v1.7\Project.toml`
  [864edb3b] + DataStructures v0.18.11
  No Changes to `C:\Users\acoap\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
   Installed NamedArrays ─── v0.9.6
   Installed Combinatorics ─ v1.0.2
    Updating `C:\Users\acoap\.julia\environments\v1.7\Project.toml`
  [86f

In [22]:
using Pkg
Pkg.add("CSV")

   Resolving package versions...
  No Changes to `C:\Users\acoap\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\acoap\.julia\environments\v1.7\Manifest.toml`


In [13]:
data = load("../../data/wage2015_subsample_inference.RData")
data = data["data"]
names(data)
println("Number of Rows : ", size(data)[1],"\n","Number of Columns : ", size(data)[2],) #rows and columns

Number of Rows : 5150
Number of Columns : 20


In [24]:
[eltype(col) for col = eachcol(data)]

20-element Vector{DataType}:
 Float64
 Float64
 Float64
 Float64
 Float64
 Float64
 Float64
 Float64
 Float64
 Float64
 Float64
 Float64
 Float64
 Float64
 Float64
 Float64
 CategoricalValue{String, UInt16}
 CategoricalValue{String, UInt8}
 CategoricalValue{String, UInt8}
 CategoricalValue{String, UInt8}

In [23]:
#Reading the CSV file into a DataFrame
#We have to set the category type for some variable
data = CSV.File("wage2015_subsample_inference.csv"; types = Dict("occ" => String,"occ2"=> String,"ind"=>String,"ind2"=>String)) |> DataFrame
println("Number of Rows : ", size(data)[1],"\n","Number of Columns : ", size(data)[2],) #rows


LoadError: UndefVarError: CSV not defined

Let's have a look at the structure of the data.

In [ ]:
***Variable description***

- occ : occupational classification
- ind : industry classification
- lwage : log hourly wage
- sex : gender (1 female) (0 male)
- shs : some high school
- hsg : High school graduated
- scl : Some College
- clg: College Graduate
- ad: Advanced Degree
- ne: Northeast
- mw: Midwest
- so: South
- we: West
- exp1: experience